In [3]:
from cProfile import run
import numpy as np
import pandas as pd
import os
from pathlib import Path
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import pandas
from tqdm import tqdm
import joblib

import torch
import os
import json
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
from sklearn.preprocessing import MinMaxScaler

import wandb
import logging
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os

logger = logging.getLogger(__name__)
wandb_logger = lambda dir, version: WandbLogger(
    name="wandb", save_dir=dir, version=version
)
csvlogger = lambda dir, version: CSVLogger(dir, name="csvlogs", version=version)
tblogger = lambda dir, version: TensorBoardLogger(dir, name="tblogs", version=version)

def get_loggers(dir,version,lis=["csv"]):
    lgrs = []
    if "wandb" in lis:
        lgrs.append(wandb_logger(dir, version))
    if "csv" in lis:
        lgrs.append(csvlogger(dir, version))
    if "tb" in lis:
        lgrs.append(tblogger(dir, version))
    return lgrs




data_dir =Path('/common/home/vk405/Projects/EnergyLab/Solar-forcast/Data/')
image_dir = Path('/common/users/vk405/EnergyLab/Data')

class Dset(Dataset):
    def __init__(self,split= 'train',data_dir='/common/home/vk405/Projects/EnergyLab/Solar-forcast/Data/',
    image_dir='/common/users/vk405/EnergyLab/Data/ProcData'):
        self.split = split

        #hardcoded dir locs 
        self.data_dir = Path(data_dir)
        self.image_dir = Path(image_dir) 
        
        base_df = pd.read_csv(f'{self.data_dir}/tgtimgs.csv')
        base_df.loc[base_df.index[base_df['Target']>=1250.0],'Target'] = 1250.0
        
        trn_df = pd.DataFrame(base_df.loc[pd.to_datetime(base_df['Date']).dt.year <= 2014])
        self.scalar = MinMaxScaler().fit(trn_df['Target'].values.reshape(-1,1))
        #splits are hardcoded as per the original paper
        if self.split == 'train':
            self.df = pd.DataFrame(base_df.loc[pd.to_datetime(base_df['Date']).dt.year <= 2014])
            
        elif self.split == 'valid':
            self.df = pd.DataFrame(base_df.loc[pd.to_datetime(base_df['Date']).dt.year == 2015])
            
        else:
            self.df = pd.DataFrame(base_df.loc[pd.to_datetime(base_df['Date']).dt.year > 2015])
        #set an upper-threshold.
        self.df.loc[self.df.index[self.df['Target']>=1250.0],'Target'] = 1250.0
        #self.df[self.df['Target']>=1300.0]['Target'] = 1300.0
        rescaled = self.scalar.transform(self.df['Target'].values.reshape(-1,1))
        self.df['Target'] = np.squeeze(rescaled)
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self,ind):
        try:
            imgs = sorted(eval(self.df.iloc[ind]['Imgs']),key = lambda x: int(x.split('/')[-1].split('.')[0]))
            target = self.df.iloc[ind]['Target']
            #image_dir = Path('/common/users/vk405/EnergyLab/Data/ProcData') 
            proc_imgs = [self.image_dir/(x.split('/')[-1].split('.')[0]+'.joblib') for x in imgs]
            proc_arrays = [joblib.load(ele) for ele in proc_imgs]
        except:
            print(ind)
            
        return (np.concatenate(proc_arrays,-1).reshape(-1,256,256))/255.0,target




def SCNN(n_ch=6):
    
    model = []
    model.append(nn.Conv2d(in_channels=n_ch, out_channels=64, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.MaxPool2d((2,2),stride=(2,2)))
    
    model.append(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.MaxPool2d((2,2),stride=(2,2)))

    model.append(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.MaxPool2d((2,2),stride=(2,2)))

    model.append(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    #model.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding="same"))
    #model.append(nn.ReLU())
    model.append(nn.MaxPool2d((2,2),stride=(2,2)))

    model.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    #model.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding="same"))
    #model.append(nn.ReLU())
    model.append(nn.MaxPool2d((2,2),stride=(2,2)))

    model.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding="same"))
    model.append(nn.ReLU())
    model.append(nn.MaxPool2d((2,2),stride=(2,2)))



    return nn.Sequential(*model)


class CombModel(pl.LightningModule):
    def __init__(self,hparams,dset=None):
        super().__init__()
        self.save_hyperparameters()

        
        if hparams.framecnt != 2:
            raise NotImplementedError

        #get the complete model
        base_model = [SCNN(hparams.framecnt*3)]
        base_model.extend([nn.Flatten(),nn.Dropout(hparams.dropoutp),nn.Linear(8192,256)])
        self.prenet = nn.Sequential(*base_model)
        #nn.Dropout(hparams.dropoutp),nn.Linear(256,1)
        self.autoregressnet = nn.Sequential(nn.Linear(hparams.lkback,hparams.lkback//2),nn.ReLU())

        self.combined_net = nn.Sequential(nn.Dropout(hparams.dropoutp),nn.Linear(256+hparams.lkback//2,1))

    def forward(self,x,lkbk_vals):
        #keep this for inference
        image_out = self.prenet(x)
        regres_out = self.autoregressnet(lkbk_vals)
        out  = self.combined_net(torch.concat([image_out,regres_out],dim=-1))
        return out

    
    def training_step(self,batch,batch_idx):
        #for training
        x,x_r,y = batch
        y_hat = torch.squeeze(self.forward(x.float(),x_r.float()))

        loss = F.l1_loss(y_hat,y.float())
        self.log("train_loss",loss,on_step=True)
        return loss

    
    def validation_step(self,batch,batch_idx):
        #for training
        x,x_r,y = batch
        y_hat = torch.squeeze(self.forward(x.float(),x_r.float()))


        loss = F.l1_loss(y_hat,y.float())
        self.log("val_loss",loss,on_step=False, on_epoch=True)
        return loss
    
    def configure_optimizers(self):
        lr = self.hparams.lr if 'lr' in self.hparams else 1e-5
        #lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False
        optimizer = torch.optim.Adam(self.parameters(), lr=lr,betas=(0.9,0.999),amsgrad=False)
        return optimizer




#new dataset

class Dset(Dataset):
    def __init__(self,split= 'train',data_dir='/common/home/vk405/Projects/EnergyLab/Solar-forcast/Data/',
    image_dir='/common/users/vk405/EnergyLab/Data/ProcData',lkback=10):
        self.split = split

        #hardcoded dir locs 
        self.data_dir = Path(data_dir)
        self.image_dir = Path(image_dir) 
        self.lkback = lkback
        
        base_df = pd.read_csv(f'{self.data_dir}/tgtimgs.csv')
        base_df.loc[base_df.index[base_df['Target']>=1250.0],'Target'] = 1250.0
        
        trn_df = pd.DataFrame(base_df.loc[pd.to_datetime(base_df['Date']).dt.year <= 2014])
        self.scalar = MinMaxScaler().fit(trn_df['Target'].values.reshape(-1,1))

        trn_msi = joblib.load(self.data_dir /'trn_msi.joblib')
        trn_msi[trn_msi>=1250] = 1250.0
        self.msi_scalar = MinMaxScaler().fit(trn_msi.reshape(-1,1))
        #splits are hardcoded as per the original paper
        if self.split == 'train':
            self.df = pd.DataFrame(base_df.loc[pd.to_datetime(base_df['Date']).dt.year <= 2014])
            self.msi = joblib.load(self.data_dir /'trn_msi.joblib')
        
            
        elif self.split == 'valid':
            self.df = pd.DataFrame(base_df.loc[pd.to_datetime(base_df['Date']).dt.year == 2015])
            self.msi = joblib.load(self.data_dir /'vld_msi.joblib')
            
        else:
            self.df = pd.DataFrame(base_df.loc[pd.to_datetime(base_df['Date']).dt.year > 2015])
            self.msi = joblib.load(self.data_dir /'tst_msi.joblib')
        #set an upper-threshold.
        self.df.loc[self.df.index[self.df['Target']>=1250.0],'Target'] = 1250.0
        #self.df[self.df['Target']>=1300.0]['Target'] = 1300.0
        rescaled = self.scalar.transform(self.df['Target'].values.reshape(-1,1))
        self.df['Target'] = np.squeeze(rescaled)
        self.msi = np.squeeze(self.msi_scalar.transform(self.msi.reshape(-1,1)))
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self,ind):
        try:
            imgs = sorted(eval(self.df.iloc[ind]['Imgs']),key = lambda x: int(x.split('/')[-1].split('.')[0]))
            target = self.df.iloc[ind]['Target']
            #image_dir = Path('/common/users/vk405/EnergyLab/Data/ProcData') 
            proc_imgs = [self.image_dir/(x.split('/')[-1].split('.')[0]+'.joblib') for x in imgs]
            proc_arrays = [joblib.load(ele) for ele in proc_imgs]
            proc_lkback = self.msi[ind*self.lkback:ind*self.lkback+10]
        except:
            print(ind)
            
        return (np.concatenate(proc_arrays,-1).reshape(-1,256,256))/255.0,proc_lkback,target



def run_model(cfg):
    pl.seed_everything(cfg.seed)
    dir = cfg.artifacts_loc
    version = str(cfg.version)
    logger_list = get_loggers(dir, version,cfg.loggers)
    cbs = []
    if "early_stop" in cfg.cbs:
        #? does'nt really work atm
        #params = cfg.model.cbs.early_stop
        earlystopcb = EarlyStopping(monitor='val_loss',mode="min",patience=3,verbose=False)
        cbs.append(earlystopcb)
    if "checkpoint" in cfg.cbs:
        store_path = dir + "ckpts/" + str(cfg.version) + "/"
        isExist = os.path.exists(store_path)
        if not isExist:
            os.makedirs(store_path)
        fname = "{epoch}-{val_loss:.2f}"
        params = cfg.checkpoint
        checkptcb = ModelCheckpoint(**params, dirpath=store_path, filename=fname)
        cbs.append(checkptcb)

    #wandb.init(project="solarforecast", config=cfg)
    if cfg.mode == 'train':
        trn_fdata = Dset(data_dir=cfg.data_dir,split=cfg.mode,image_dir= cfg.image_dir,\
               lkback=cfg.lkback)
        vld_fdata = Dset(data_dir=cfg.data_dir,split='valid',image_dir= cfg.image_dir,lkback=cfg.lkback)

        val_loader = DataLoader(vld_fdata,\
            batch_size=cfg.batch_size,shuffle=False,num_workers=4,pin_memory=True)
        train_loader = DataLoader(trn_fdata,\
            batch_size=cfg.batch_size,shuffle=True,num_workers=4,pin_memory=True)
            
        hparams = cfg    
        net = CombModel(hparams)
        trainer = pl.Trainer(
            logger=logger_list,callbacks=cbs, gpus=[0,1,2,3],deterministic=True, **cfg.trainer
        )
        trainer.fit(net,train_dataloaders=train_loader,val_dataloaders=val_loader)
        return trainer
        #trainer.tune(net,train_loader)
            
    else:
        pass
    

## Preprocessing

In [10]:
trndset = Dset()


In [34]:
trndset.df.iloc[0]

Date                                             2012-01-01
MST                                                   07:40
Imgs      ['/common/users/vk405/EnergyLab/Data/20120101/...
Target                                              0.02127
Name: 0, dtype: object

In [3]:
df_svd = pd.read_csv(os.path.join(data_dir, 'SRRL_measurement_timeseries.csv'))

In [12]:
df_svd.head()

,Date,MST,GHI,DateTime
0,2012-01-01,07:25,10.991,2012-01-01 07:25
1,2012-01-01,07:26,12.329,2012-01-01 07:26
2,2012-01-01,07:27,13.698,2012-01-01 07:27
3,2012-01-01,07:28,15.007,2012-01-01 07:28
4,2012-01-01,07:29,15.516,2012-01-01 07:29


In [16]:
smpl = ' '.join(['2012-01-01','07:40'])	

In [27]:
df_svd[df_svd['DateTime'] == smpl].index.tolist()[0]


15

In [4]:
def extract(all_data,ext_for,lkbk=10):
    ext_vals = []
    for i in tqdm(range(len(ext_for))):
        dt = ' '.join([ext_for.iloc[i]['Date'],ext_for.iloc[i]['MST']])
        ind = all_data[all_data['DateTime'] == dt].index.tolist()[0]
        ext_vals.append(all_data.iloc[ind-lkbk+1:ind+1]['GHI'].values)
    return np.concatenate(ext_vals)




In [37]:
trn_msi = extract(df_svd,trndset.df)

  7%|▋         | 3925/58531 [05:40<1:18:51, 11.54it/s]

In [ ]:
joblib.dump(trn_msi,data_dir/'trn_msi.joblib')

In [ ]:
vlddset = Dset(split='valid')

vld_msi = extract(df_svd,vlddset.df)
joblib.dump(vld_msi,data_dir/'vld_msi.joblib')

array([16.033, 17.699, 19.783, 19.342, 18.155, 18.422, 18.262, 17.904,
       18.133, 18.962, 20.072])

In [ ]:
tstdset = Dset(split='test')

tst_msi = extract(df_svd,tstdset.df)
joblib.dump(tst_msi,data_dir/'tst_msi.joblib')

In [5]:
trn_msi = joblib.load(data_dir/'trn_msi.joblib')

In [23]:
trn_msi[trn_msi>=1250] = 1250.0

In [25]:
s = MinMaxScaler().fit(trn_msi.reshape(-1,1))

In [29]:
s.transform([[1200]])

array([[0.96]])

In [14]:
len(trn_msi)

585310

In [15]:
trn_msi[0*10:0*10+10]

array([17.699, 19.783, 19.342, 18.155, 18.422, 18.262, 17.904, 18.133,
       18.962, 20.072])

In [18]:
trn_msi.max(),trn_msi.min()

(1587.36, 0.0)

### Building a joint model

In [36]:
#testing

dset = Dset()
dloader = DataLoader(dset,\
            batch_size=8,shuffle=False)
x,x_r,y = None,None,None
for batch in dloader:
    x,x_r,y = batch
    print(x.shape,x_r.shape,y.shape)
    break

torch.Size([8, 6, 256, 256]) torch.Size([8, 10]) torch.Size([8])


In [34]:
combmodel = CombModel(cfg)

In [37]:
out = combmodel(x.float(),x_r.float())

In [39]:
out.shape

torch.Size([8, 1])

In [5]:
    from argparse import Namespace
    cfg = Namespace(
        version = 'combtrail',
        artifacts_loc = "/common/home/vk405/Projects/EnergyLab/Solar-forcast/artifacts/",
        data_dir = "/common/home/vk405/Projects/EnergyLab/Solar-forcast/Data/",
        image_dir = "/common/users/vk405/EnergyLab/Data/ProcData/",
        mode = 'train',
        loggers = ["csv"],
        seed = 0,
        cbs = ["checkpoint"],
        trainer = {'log_every_n_steps': 50,
        'max_epochs': 40},
        checkpoint = {"save_top_k": 5,
        "monitor": "val_loss","mode":"min"},
        dropoutp=0.2,
        framecnt=2,
        lr = 1.5e-5,
        batch_size=64,
        lkback = 10
    )

In [7]:
def infer(model,loader):
    preds_lis = []
    truelabels = []
    with torch.no_grad():
        for x,x_r,y in tqdm(test_loader):
            preds = model(x.float(),x_r.float())
            preds_lis.append(preds.squeeze().cpu().numpy())
            truelabels.append(y.cpu().numpy())
    return np.concatenate(preds_lis,axis=0),np.concatenate(truelabels,axis=0)

In [8]:
#inference

#trained model
net = CombModel(cfg)
weight_loc = '/common/home/vk405/Projects/EnergyLab/Solar-forcast/artifacts/ckpts/combtrail/epoch=39-val_loss=0.08.ckpt'
trnd_net = net.load_from_checkpoint(weight_loc)

test_dset = Dset(split= 'test')
test_loader = DataLoader(test_dset,\
            batch_size=64,shuffle=False,num_workers=4,pin_memory=True)


    


In [10]:
p,g = infer(trnd_net,test_loader)
infered_vals = {'Date':test_dset.df['Date'].values,'MST':test_dset.df['MST'].values,
    'pred':p,'ground':g,'Target':test_dset.df['Target'].values}
pd.DataFrame(infered_vals).to_csv(data_dir/'infered_vals_comb.csv',index=False)

100%|██████████| 436/436 [39:19<00:00,  5.41s/it]


In [16]:
tst_preds = pd.DataFrame(infered_vals)
#tst_preds['abs_error'] = np.abs(tst_preds['ground']-tst_preds['pred'])
tst_preds['error'] = tst_preds['pred']-tst_preds['ground']

In [17]:

small,med = tst_preds['Target'].quantile(0.33),tst_preds['Target'].quantile(0.66)

cloudy = tst_preds[tst_preds['Target']<=small]
normal = tst_preds[(tst_preds['Target']>small) & (tst_preds['Target']<=med)]
sunny = tst_preds[(tst_preds['Target']>med)]

In [18]:
# yes cloudy is bad(can do hypothesis testing for better confirmation.)
cloudy['error'].abs().mean(),sunny['error'].abs().mean()

(0.11896533868615822, 0.0629888550466521)

In [19]:
cloudy['error'].abs().mean(),normal['error'].abs().mean()

(0.11896533868615822, 0.08061535536523695)

In [21]:
#over-prediction vs underprediction(looks not so convincing.)
over_preds = tst_preds[tst_preds['error']>0.0]
under_preds = tst_preds[tst_preds['error']<=0.0]

In [25]:
over_preds['error'].abs().mean(),under_preds['error'].abs().mean()

(0.09761838431070016, 0.06821019487593272)

In [26]:
#
tst_preds['error'].abs().mean()

0.08727852826239703